In [418]:
import pandas as pd
from pathlib import Path 
import csv 
from sys import stderr 
from collections import defaultdict

In [419]:
movies = pd.read_csv(Path.cwd()/'../data/movies/top500movies.csv')

In [420]:
movies.head(100)

,Title,Relased_Year,Genre,IMDB_Rating
0,Avatarâ,2009,Action|Adventure|Fantasy|Sci-Fi,7.9
1,Jurassic Worldâ,2015,Action|Adventure|Sci-Fi|Thriller,7.0
2,Titanicâ,1997,Drama|Romance,7.7
3,Star Wars: Episode Iv - A New Hopeâ,1977,Action|Adventure|Fantasy|Sci-Fi,8.7
4,E.T. The Extra-Terrestrialâ,1982,Family|Sci-Fi,7.9
...,...,...,...,...
95,Home Alone 2: Lost In New Yorkâ,1992,Adventure|Comedy|Family,6.6
96,Tootsieâ,1982,Comedy|Drama|Romance,7.4
97,The Sound Of Musicâ,1965,Biography|Drama|Family|Musical|Romance,8.0
98,Twilightâ,2008,Drama|Fantasy|Romance,5.2


1. kreirati metodu koja vraca pandas framework koji pokazuje koliko filmova kog žanra postoji u uzorku.

1.1. Prvo kreiramo metodu koja čita iz .csv fajla i vraća svaku ntorku kao dictionary, čiji je key naziv kolone, a value vrednost na toj koloni

In [421]:
def read_as_dict_from_csv(fpath):
    try:
        with open(fpath, 'r') as fobj:
            return list(csv.DictReader(fobj))
    except csv.Error as err:
        stderr.write(f"Error while trying to read file. {err}\n")
        
    

1.2. Sada testiramo metodu, tj. čitamo sa tog fajla i ispisujemo vrednosti liste dictionaryja.

In [422]:
list_of_movies = read_as_dict_from_csv(Path.cwd()/'../data/movies/top500movies.csv')
for movie in list_of_movies:
    print(movie)

{'Title': 'AvatarÃ¢Â\xa0', 'Relased_Year': '2009', 'Genre ': 'Action|Adventure|Fantasy|Sci-Fi', 'IMDB_Rating': '7.9'}
{'Title': 'Jurassic WorldÃ¢Â\xa0', 'Relased_Year': '2015', 'Genre ': 'Action|Adventure|Sci-Fi|Thriller', 'IMDB_Rating': '7'}
{'Title': 'TitanicÃ¢Â\xa0', 'Relased_Year': '1997', 'Genre ': 'Drama|Romance', 'IMDB_Rating': '7.7'}
{'Title': 'Star Wars: Episode Iv - A New HopeÃ¢Â\xa0', 'Relased_Year': '1977', 'Genre ': 'Action|Adventure|Fantasy|Sci-Fi', 'IMDB_Rating': '8.7'}
{'Title': 'E.T. The Extra-TerrestrialÃ¢Â\xa0', 'Relased_Year': '1982', 'Genre ': 'Family|Sci-Fi', 'IMDB_Rating': '7.9'}
{'Title': 'The AvengersÃ¢Â\xa0', 'Relased_Year': '2012', 'Genre ': 'Action|Adventure|Sci-Fi', 'IMDB_Rating': '8.1'}
{'Title': 'The AvengersÃ¢Â\xa0', 'Relased_Year': '2012', 'Genre ': 'Action|Adventure|Sci-Fi', 'IMDB_Rating': '8.1'}
{'Title': 'The Lion KingÃ¢Â\xa0', 'Relased_Year': '1994', 'Genre ': 'Adventure|Animation|Drama|Family|Musical', 'IMDB_Rating': '8.5'}
{'Title': 'Star Wars: Ep

1.3. Sada od kreiramo metodu koja prima listu dictionaryja, prolazi kroz nju, vraća listu vrednosti žanrova za svaki film i pravi dictionary čije su value vrednosti liste ntorki koje pripadaju tom žanru. Kasnije pravimo još jedan dictionary čije su vrednosti integeri, tj. broj filmova tog žanra. Metoda vraća dictionary koji je sortiran opadajuće po vrednostima broja filmova u jednom žanru. 

In [423]:
def genre_movie_count_dict(movies):
    dict_genre_movies = defaultdict(list)
    dict_genres_number = defaultdict(int)
    for movie in movies:
        genres = movie['Genre '].split('|')
        for genre in genres:
            #print(genre)
            dict_genre_movies[str(genre)].append(dict(movie))
            #print(dict_genre_movies[str(genre)])
    for genre, movies in dict_genre_movies.items():
        dict_genres_number[genre] = len(movies)
    return dict(sorted(dict_genres_number.items(),key=lambda elem:elem[1], reverse=True))

1.4. Pravimo DataFrame koji ima dve kolone - žanr i broj filmova tog žanra.

In [424]:
genre_count_dict = genre_movie_count_dict(list_of_movies)
# for genre,movies in genre_count_dict.items():
#     print(genre,'--->',str(movies))
genre_movies_frame = pd.DataFrame(genre_count_dict.items())
genre_movies_frame.columns = ['Genre', 'Total_Count']
genre_movies_frame

,Genre,Total_Count
0,Comedy,216
1,Drama,181
2,Adventure,174
3,Action,156
4,Romance,109
5,Thriller,108
6,Family,108
7,Fantasy,106
8,Sci-Fi,92
9,Crime,59


2. Napisati metodu koja pronalazi za svaku godinu i za svaki žanr prosečnu ocenu imdb. Metoda vraća sortirani dictionary godina, žanr - prosečni imdb  

2.1. Treba nam jedan dictionary - key je godina, a value je lista filmova koji su objavljeni te godine. Drugi dictionary ima key - žanr, a value je lista filmova koji su snimljeni po tom žanru. 

In [425]:
def get_avg_rating(movies):
    dict_year_genre_movies = defaultdict(list)
    dict_year_genre_score = defaultdict(float)
    for movie in movies:
        genres = movie['Genre '].split('|')
        for genre in genres:
            key = movie['Relased_Year'],genre
            dict_year_genre_movies[key].append(movie)
    #for key, val in dict_year_genre_movies.items():
    #    print(key,'-->',val)
    for key, val in dict_year_genre_movies.items(): # prolazimo kroz dictionary godina - zanr - lista filmova
        #print(val)
        sum_of_ratings = 0                               
        count_movies = len(val)                      # broj filmova za tu godinu i taj zanr
        for it_movie in range(len(val)):             # prolazimo kroz listu filmova za tu godinu i za taj zanr
            #print(val[it_movie]['IMDB_Rating'])
            sum_of_ratings += float(val[it_movie]['IMDB_Rating'])      # pravimo sumu ocena za tu godinu i taj zanr
            #print(sum_of_ratings)
        dict_year_genre_score[key] = sum_of_ratings/count_movies       # u novi recnik dodajemo prosecnu ocenu
    return dict(sorted(dict_year_genre_score.items(), key = lambda elem: elem[1], reverse=True)) # soritramo po vrednosti prosecne ocene

In [426]:
dict_year_genre_score = get_avg_rating(list_of_movies)
df_year_genre_avg_score = pd.DataFrame(dict_year_genre_score.items())
df_year_genre_avg_score.columns = ['Year - Genre', 'Average_score']
df_year_genre_avg_score

,Year - Genre,Average_score
0,"(1972, Crime)",9.2
1,"(1972, Drama)",9.2
2,"(1993, Biography)",8.9
3,"(1993, History)",8.9
4,"(1980, Fantasy)",8.8
...,...,...
537,"(2008, Music)",4.5
538,"(2011, Fantasy)",4.4
539,"(2012, Horror)",4.2
540,"(2011, Music)",3.0


- Sada ćemo posmatrati fajl imdb_top_1000_cleared_data.csv

In [428]:
top1000movies = pd.read_csv(Path.cwd()/'../data/movies/top1000original.csv')

In [429]:
stars = top1000movies[['Star1','Star2','Star3','Star4']]
stars

,Star1,Star2,Star3,Star4
0,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler
1,Marlon Brando,Al Pacino,James Caan,Diane Keaton
2,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
3,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton
4,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler
...,...,...,...,...
995,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen
996,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker
997,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed
998,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix


In [430]:
list_of_stars = list()
for star in stars.values:
    list_of_stars.append(star)
    

In [431]:
# for elem in list_of_stars:
#     str_star = str(elem)
#     print(str_star)
series = pd.Series(list_of_stars, name='Actors') 
top1000movies = pd.concat([top1000movies, series], axis=1)

In [432]:
top1000movies =top1000movies.drop(columns=['Star1','Star2','Star3','Star4'])

In [433]:
top1000movies

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,No_of_Votes,Gross,Actors
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,2343110,"28,341,469","[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,1620367,"134,966,411","[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,2303232,"534,858,444","[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,1129952,"57,300,000","[Al Pacino, Robert De Niro, Robert Duvall, Dia..."
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,689845,"4,360,000","[Henry Fonda, Lee J. Cobb, Martin Balsam, John..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,166544,NaN,"[Audrey Hepburn, George Peppard, Patricia Neal..."
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,34075,NaN,"[Elizabeth Taylor, Rock Hudson, James Dean, Ca..."
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,43374,"30,500,000","[Burt Lancaster, Montgomery Clift, Deborah Ker..."
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,26471,NaN,"[Tallulah Bankhead, John Hodiak, Walter Slezak..."


3.0. Sačuvaćemo ovaj fajl u neki csv dokument.

In [434]:
# def write_to_csv(fpath,movies,fieldnames, delimeter):
#     try:
#         with open(fpath,'w') as fobj:
#             csv_writer = csv.DictWriter(fobj, delimiter=delimeter,filednames=fieldnames)
#             csv_writer.writeheader()
#             for movie in movies:
#                 csv_writer.writerow(movie)
#     except csv.Error:
#         stderr.write("Error while trying to write into file.")
def write_to_csv_as_dict(data, fpath, fieldnames, delimiter=','):
    try:
        with open(fpath, 'w') as fobj:
            csv_writer = csv.DictWriter(fobj, delimiter=delimiter, fieldnames=fieldnames)
            csv_writer.writeheader()
            for data_row in data:
                csv_writer.writerow(data_row)
    except csv.Error as err:
        stderr.write(f"CSV specific error occurred while trying to write to file {fpath}:\n{err}\n")
    except OSError as err:
        stderr.write(f"OS error occurred while trying to write to file {fpath}:\n{err}\n")


AttributeError: 'str' object has no attribute 'keys'

In [435]:
list_of_stars_str = list()
for element in top1000movies['Actors']:
    actors = ''
    #print(element)
    actors =', '.join([actor for actor in element])
    list_of_stars_str.append(actors)
    print(actors)     

Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler
Marlon Brando, Al Pacino, James Caan, Diane Keaton
Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine
Al Pacino, Robert De Niro, Robert Duvall, Diane Keaton
Henry Fonda, Lee J. Cobb, Martin Balsam, John Fiedler
Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom
John Travolta, Uma Thurman, Samuel L. Jackson, Bruce Willis
Liam Neeson, Ralph Fiennes, Ben Kingsley, Caroline Goodall
Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page, Ken Watanabe
Brad Pitt, Edward Norton, Meat Loaf, Zach Grenier
Elijah Wood, Ian McKellen, Orlando Bloom, Sean Bean
Tom Hanks, Robin Wright, Gary Sinise, Sally Field
Clint Eastwood, Eli Wallach, Lee Van Cleef, Aldo Giuffrè
Elijah Wood, Ian McKellen, Viggo Mortensen, Orlando Bloom
Lilly Wachowski, Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss
Robert De Niro, Ray Liotta, Joe Pesci, Lorraine Bracco
Mark Hamill, Harrison Ford, Carrie Fisher, Billy Dee Williams
Jack Nicholson, Louise Fletch

In [436]:
len(list_of_stars_str)
top1000movies = pd.concat([top1000movies,pd.Series(list_of_stars_str, name='Stars')], axis=1)

In [437]:
top1000movies = top1000movies.drop(columns=['Actors'])

In [438]:
top1000movies

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,No_of_Votes,Gross,Stars
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,2343110,"28,341,469","Tim Robbins, Morgan Freeman, Bob Gunton, Willi..."
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,1620367,"134,966,411","Marlon Brando, Al Pacino, James Caan, Diane Ke..."
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,2303232,"534,858,444","Christian Bale, Heath Ledger, Aaron Eckhart, M..."
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,1129952,"57,300,000","Al Pacino, Robert De Niro, Robert Duvall, Dian..."
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,689845,"4,360,000","Henry Fonda, Lee J. Cobb, Martin Balsam, John ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,166544,NaN,"Audrey Hepburn, George Peppard, Patricia Neal,..."
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,34075,NaN,"Elizabeth Taylor, Rock Hudson, James Dean, Car..."
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,43374,"30,500,000","Burt Lancaster, Montgomery Clift, Deborah Kerr..."
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,26471,NaN,"Tallulah Bankhead, John Hodiak, Walter Slezak,..."


Sada cemo to da pročitamo.

3.1. Potrebno je skladištiti sve podatke iz data frame - a u jedan fajl. Paziti da se novi dictionary inicijalizuje sa res_dict = dict() tek pred drugu for petlju (da ne bude 1000 istih filmova - 1000 1000. filmova)

In [439]:
res_list=list()
columns = top1000movies.columns.to_list()
#print(len(res_list))
for movie in top1000movies.values:
    res_dict = dict()
    for key, val in zip(columns, movie):
        res_dict[key]=val
    res_list.append(res_dict)
    #print(res_dict)
fpath = Path.cwd()/'../data/top_1000_movies_corrected_values_2.csv'
fieldnames = top1000movies.columns.to_list()
write_to_csv_as_dict(res_list,fpath,fieldnames)

OS error occurred while trying to write to file C:\Users\Korisnik\DataspellProjects\kolokvijum1\code\..\data\top_1000_movies_corrected_values_2.csv:
[Errno 13] Permission denied: 'C:\\Users\\Korisnik\\DataspellProjects\\kolokvijum1\\code\\..\\data\\top_1000_movies_corrected_values_2.csv'


In [440]:
movies_list = read_as_dict_from_csv(Path.cwd()/'../data/movies/top_1000_movies_corrected_values_2.csv')

In [441]:
for movie in movies_list:
    print(movie)

{'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg', 'Series_Title': 'The Shawshank Redemption', 'Released_Year': '1994', 'Certificate': 'A', 'Runtime': '142 min', 'Genre': 'Drama', 'IMDB_Rating': '9.3', 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'Meta_score': '80.0', 'Director': 'Frank Darabont', 'No_of_Votes': '2343110', 'Gross': '28,341,469', 'Stars': 'Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler'}
{'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY98_CR1,0,67,98_AL_.jpg', 'Series_Title': 'The Godfather', 'Released_Year': '1972', 'Certificate': 'A', 'Runtime': '175 min', 'Genre': 'Crime, Drama', 'IMDB_Rating': '9.2', 'Overview': "An organized crime dynasty's aging patriarch transfers c

3.2. Za svakog glumca pronaći prosečan imdb, maximalni imdb, broj filmova u kojima je glumio, kao i najčešći žanr u kojem glumi. Sve sortirati po prosečnom imdb-u opadajuće

In [442]:
def get_actors_statistics(movies):
    dict_actor_movies=defaultdict(list)
    dict_actor_movie_count=defaultdict(int)
    dict_actor_avg_imdb = defaultdict(float)
    dict_actor_max_imdb = defaultdict(float)
    actors_list = list()  # Ime_Prezime, Broj_filmova, Prosecni_IMDB, Maximalni_IMDB
    for movie in movies:
        actors = movie['Stars'].split(', ')
        for actor in actors:
            dict_actor_movies[actor].append(movie)
    for key, val in dict_actor_movies.items():
        dict_actor_movie_count[key] = len(val)
        sum_of_imdbs=0
        max_imdb = 0
        for movie in val:
            sum_of_imdbs += float(movie['IMDB_Rating'])
            if float(movie['IMDB_Rating'])>max_imdb:
                max_imdb = float(movie['IMDB_Rating'])
        dict_actor_avg_imdb[key] = sum_of_imdbs/len(val)
        dict_actor_max_imdb[key] = max_imdb
    for key, val in dict_actor_movies.items():
        supplementary_dict = dict()
        supplementary_dict['Firstname_Lastname'] = key
        supplementary_dict['Movie_Count'] = dict_actor_movie_count[key]
        supplementary_dict['Average_IMDB'] = dict_actor_avg_imdb[key]
        supplementary_dict['Max_IMDB'] = dict_actor_max_imdb[key]
        actors_list.append(supplementary_dict)
    return sorted(actors_list, key=lambda actor:actor['Average_IMDB'], reverse=True)

Ovde ćemo testirati funkciju get_actors_statistics(movies):

In [443]:
actors_stats = get_actors_statistics(movies_list)
for actor in actors_stats:
    print(actor)

{'Firstname_Lastname': 'Bob Gunton', 'Movie_Count': 1, 'Average_IMDB': 9.3, 'Max_IMDB': 9.3}
{'Firstname_Lastname': 'William Sadler', 'Movie_Count': 1, 'Average_IMDB': 9.3, 'Max_IMDB': 9.3}
{'Firstname_Lastname': 'Aaron Eckhart', 'Movie_Count': 1, 'Average_IMDB': 9.0, 'Max_IMDB': 9.0}
{'Firstname_Lastname': 'John Travolta', 'Movie_Count': 1, 'Average_IMDB': 8.9, 'Max_IMDB': 8.9}
{'Firstname_Lastname': 'Caroline Goodall', 'Movie_Count': 1, 'Average_IMDB': 8.9, 'Max_IMDB': 8.9}
{'Firstname_Lastname': 'Elijah Wood', 'Movie_Count': 3, 'Average_IMDB': 8.8, 'Max_IMDB': 8.9}
{'Firstname_Lastname': 'Elliot Page', 'Movie_Count': 1, 'Average_IMDB': 8.8, 'Max_IMDB': 8.8}
{'Firstname_Lastname': 'Meat Loaf', 'Movie_Count': 1, 'Average_IMDB': 8.8, 'Max_IMDB': 8.8}
{'Firstname_Lastname': 'Zach Grenier', 'Movie_Count': 1, 'Average_IMDB': 8.8, 'Max_IMDB': 8.8}
{'Firstname_Lastname': 'Sean Bean', 'Movie_Count': 1, 'Average_IMDB': 8.8, 'Max_IMDB': 8.8}
{'Firstname_Lastname': 'Sally Field', 'Movie_Count':

3.3. Sada ćemo napraviti dataframe od ovih podataka.

In [444]:
actors_stats_df = pd.DataFrame(actors_stats)

In [445]:
actors_stats_df

,Firstname_Lastname,Movie_Count,Average_IMDB,Max_IMDB
0,Bob Gunton,1,9.3,9.3
1,William Sadler,1,9.3,9.3
2,Aaron Eckhart,1,9.0,9.0
3,John Travolta,1,8.9,8.9
4,Caroline Goodall,1,8.9,8.9
...,...,...,...,...
2704,William Bendix,1,7.6,7.6
2705,Robert Donat,1,7.6,7.6
2706,Madeleine Carroll,1,7.6,7.6
2707,Lucie Mannheim,1,7.6,7.6


4. Napisati metodu koja vraća top 50 najdužih drama iz 21. veka sa Metascore rejtingom preko 60. Prikazati i Certificate.

In [446]:
def get_the_longest_drama_movies_21st_imdb60plus(movies):
    drama_movies_list=list()
    movie_duration_dict = dict()
    movie_metascore = dict()
    movie_certifikate = dict()
    result_list = list()
    for movie in movies:
        genres = movie['Genre'].split(', ')
        if not 'Drama' in genres:
            continue
        if not float(movie['Meta_score'])>60:
            continue 
        try:
            if not int(movie['Released_Year'])>=2000:
                continue
        except ValueError:
            continue
        drama_movies_list.append(movie)
    for drama_movie in drama_movies_list:
        duration = int(drama_movie['Runtime'].split(' ')[0])
        movie_duration_dict[drama_movie['Series_Title']] = duration 
        movie_metascore[drama_movie['Series_Title']] = drama_movie['Meta_score']
        movie_certifikate[drama_movie['Series_Title']] = drama_movie['Certificate']
        temporary_dict = dict()
        temporary_dict['Title'] = drama_movie['Series_Title']
        temporary_dict['Certificate'] = movie_certifikate[drama_movie['Series_Title']]
        temporary_dict['Metascore'] =movie_metascore[drama_movie['Series_Title']]
        temporary_dict['Duration'] = movie_duration_dict[drama_movie['Series_Title']]
        result_list.append(temporary_dict)
    return sorted(result_list,key=lambda elem:elem['Duration'], reverse=True)

In [447]:
movies_statistics =get_the_longest_drama_movies_21st_imdb60plus(movies_list)
# for elem in movies_statistics:
#     print(elem)
movies_statistics_df = pd.DataFrame(movies_statistics)

In [448]:
movies_statistics_df

,Title,Certificate,Metascore,Duration
0,Lagaan: Once Upon a Time in India,U,84.0,224
1,The Irishman,R,94.0,209
2,The Lord of the Rings: The Return of the King,U,94.0,201
3,Kis Uykusu,nan,88.0,196
4,Veer-Zaara,U,67.0,192
...,...,...,...,...
268,Bin-jip,U,72.0,88
269,Mandariinid,nan,73.0,87
270,Once,R,88.0,86
271,Before Sunset,R,90.0,80


5. Za svaki žanr odrediti prosečno trajanje filma i odrediti žanrove sa najvećom dužinom filma. 

In [449]:
def get_genres_and_avg_durations(movies):
    genres_movies_dict=defaultdict(list)
    genres_durations = defaultdict(float)
    for movie in movies:
        genres = movie['Genre'].split(', ')
        for genre in genres:
            genres_movies_dict[genre].append(movie)
    #sum_duration = 0
    #len_movies = 0
    for key, val in genres_movies_dict.items():
        sum_duration = 0
        len_movies = 0
        for movie in val:
            sum_duration += float(movie['Runtime'].split(' ')[0])
        len_movies = len(val)
        # print(sum_duration)
        # print(len(val))
        # print(sum_duration/len_movies)
        genres_durations[key] = sum_duration/len_movies
    return dict(sorted(genres_durations.items(), key=lambda el:el[1], reverse=True))

In [450]:
movies_lst=read_as_dict_from_csv(Path.cwd()/'../data/movies/top_1000_movies_corrected_values_2.csv')
rating_genres = get_genres_and_avg_durations(movies_lst)
for key, val in rating_genres.items():
    print(key,'-->',val)

History --> 143.55357142857142
Musical --> 140.2941176470588
Biography --> 135.6697247706422
Western --> 134.6
Sport --> 132.6315789473684
War --> 132.0392156862745
Action --> 126.72486772486772
Drama --> 126.70165745856353
Adventure --> 125.5
Crime --> 123.4401913875598
Mystery --> 121.14141414141415
Sci-Fi --> 120.80597014925372
Music --> 120.74285714285715
Romance --> 119.928
Thriller --> 119.64233576642336
Fantasy --> 114.77272727272727
Family --> 114.14285714285714
Comedy --> 111.63948497854078
Horror --> 102.5
Film-Noir --> 100.73684210526316
Animation --> 99.58536585365853


In [451]:
pd.DataFrame(rating_genres.items(), columns=['Genres','Durations'])

,Genres,Durations
0,History,143.553571
1,Musical,140.294118
2,Biography,135.669725
3,Western,134.600000
4,Sport,132.631579
5,War,132.039216
6,Action,126.724868
7,Drama,126.701657
8,Adventure,125.500000
9,Crime,123.440191
